<a href="https://colab.research.google.com/github/mateuszmilewski/quickCsvTest001/blob/main/pyspark001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## just some pip to install the spark for the python

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=193d4ada78127919758c8720144e4e530173592b6839b32417ac9c8e4e6e0e9d
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# pyspark workspace

In [ ]:
## some hello world exmaple

In [3]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("HelloWorld").getOrCreate()

# Create a Resilient Distributed Dataset (RDD)
data = ["Hello", "World"]
rdd = spark.sparkContext.parallelize(data)

# Perform a map operation on the RDD
result = rdd.map(lambda x: x + " PySpark!").collect()

# Print the result
for item in result:
    print(item)

# Stop the Spark session
spark.stop()


Hello PySpark!
World PySpark!


## to generate csv file - lets check if this work!

wow this is really good - below code just generated for me the file

In [4]:
import csv
import random

# Sample data
companies = [
    "Acme Corporation",
    "Widget Innovators",
    "Global Ventures",
    "Tech Solutions Ltd",
    "Quantum Industries",
    "NexTech Systems",
    "Data Dynamics",
    "Alpha Innovations",
    "Infinite Insights",
    "Bright Ideas Co."
]

# Generate random data
rows = []
for _ in range(10):
    company = random.choice(companies)
    revenue = random.randint(500000, 3000000)
    employees = random.randint(10, 100)
    rows.append([company, revenue, employees])

# Write to CSV file
with open("random_business_data.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Company Name", "Revenue (in USD)", "Number of Employees"])
    writer.writerows(rows)

print("CSV file 'random_business_data.csv' created successfully!")


CSV file 'random_business_data.csv' created successfully!


## non becuase I have this csv file here - lets try to see it through pyspark

In [5]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("Read CSV Example").getOrCreate()

# Read the CSV file into a DataFrame
df = spark.read.csv("random_business_data.csv", header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()

# Stop the Spark session
spark.stop()


+------------------+----------------+-------------------+
|      Company Name|Revenue (in USD)|Number of Employees|
+------------------+----------------+-------------------+
|Tech Solutions Ltd|         1009180|                 74|
|  Bright Ideas Co.|         2530429|                 29|
| Alpha Innovations|         1065817|                 85|
| Infinite Insights|         2060767|                 67|
|  Bright Ideas Co.|          549265|                 53|
|  Acme Corporation|         2361316|                 20|
| Widget Innovators|         1823113|                 70|
| Alpha Innovations|         1116789|                 45|
|   Global Ventures|         1964298|                 61|
|  Acme Corporation|         1349683|                 29|
+------------------+----------------+-------------------+



Explanation:

We use the avg and max functions from pyspark.sql.functions to compute the average revenue and maximum number of employees, respectively.
The results are printed to the console.
Feel free to explore more statistical analyses or customize the code further based on your requirements! 📊🚀✨

In [14]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, max, min

# Create a Spark session
spark = SparkSession.builder.appName("Business Statistics").getOrCreate()

# Read the CSV file into a DataFrame
df = spark.read.csv("random_business_data.csv", header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()

# Calculate average revenue
avg_revenue = df.select(avg("Revenue (in USD)")).collect()[0][0]
print(f"Average Revenue: ${avg_revenue:.2f}")

# Calculate maximum number of employees
max_employees = df.select(max("Number of Employees")).collect()[0][0]
print(f"Maximum Employees: {max_employees}")

# Calculate avr number of employees
min_employees = df.select(min("Number of Employees")).collect()[0][0]
print(f"average number of Employees: {min_employees}")

# Stop the Spark session
spark.stop()


+------------------+----------------+-------------------+
|      Company Name|Revenue (in USD)|Number of Employees|
+------------------+----------------+-------------------+
|Tech Solutions Ltd|         1009180|                 74|
|  Bright Ideas Co.|         2530429|                 29|
| Alpha Innovations|         1065817|                 85|
| Infinite Insights|         2060767|                 67|
|  Bright Ideas Co.|          549265|                 53|
|  Acme Corporation|         2361316|                 20|
| Widget Innovators|         1823113|                 70|
| Alpha Innovations|         1116789|                 45|
|   Global Ventures|         1964298|                 61|
|  Acme Corporation|         1349683|                 29|
+------------------+----------------+-------------------+

Average Revenue: $1583065.70
Maximum Employees: 85
average number of Employees: 20


In [19]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Create a Spark session
spark = SparkSession.builder.appName("Read CSV Example").getOrCreate()

# Read the CSV file into a DataFrame
df = spark.read.csv("random_business_data.csv", header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()


header_col = 'Company Name'
cols_minus_header = df.columns
cols_minus_header.remove(header_col)


# this is strange!
##########################################
df_transposed = (
    df.groupBy()
    .pivot(header_col)
    .agg(F.first(F.array(*cols_minus_header)))
    .withColumn(header_col, F.array(*map(F.lit, cols_minus_header)))
)
##########################################



print('Quasi Transpose: ')

# Show the transposed DataFrame
df_transposed.show(truncate=False)

# Stop the Spark session
spark.stop()

+------------------+----------------+-------------------+
|      Company Name|Revenue (in USD)|Number of Employees|
+------------------+----------------+-------------------+
|Tech Solutions Ltd|         1009180|                 74|
|  Bright Ideas Co.|         2530429|                 29|
| Alpha Innovations|         1065817|                 85|
| Infinite Insights|         2060767|                 67|
|  Bright Ideas Co.|          549265|                 53|
|  Acme Corporation|         2361316|                 20|
| Widget Innovators|         1823113|                 70|
| Alpha Innovations|         1116789|                 45|
|   Global Ventures|         1964298|                 61|
|  Acme Corporation|         1349683|                 29|
+------------------+----------------+-------------------+

Quasi Transpose: 
+----------------+-----------------+----------------+---------------+-----------------+------------------+-----------------+---------------------------------------+
|Acm